In [1]:
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
stocks = pd.read_csv("C:\\College\\College\\Quantum Computing\\World-Stock-Prices-Dataset.csv")
select_stock = stocks[stocks['Brand_Name'] == 'apple']

In [3]:
myStock = select_stock[['Date', 'Close', 'Volume']].copy()
myStock = myStock.rename(columns={'Date': 'ds', 'Close': 'y', 'Volume': 'volume'})

In [4]:
myStock['ds'] = pd.to_datetime(myStock['ds'], errors='coerce', utc=True).dt.tz_localize(None)
myStock['ds'] = myStock['ds'].map(pd.Timestamp.timestamp)
print(myStock['ds'].min(), myStock['ds'].max())
myStock['y'] = myStock['y'].astype(float)
myStock.dropna(inplace=True)

946875600.0 1751515200.0


In [5]:
myStock.shape

(6470, 3)

In [6]:
myStock.reset_index()
myStock.head()

,ds,y,volume
4,1.751515e+09,213.550003,34697317.0
86,1.751515e+09,213.550003,34955800.0
112,1.751429e+09,212.440002,67893400.0
204,1.751342e+09,207.820007,78673300.0
236,1.751256e+09,205.169998,91612700.0


In [ ]:
numQubits = 2
dev = qml.device("lightning.gpu", wires=numQubits)

@qml.qnode(device=dev)
def qkernel_circuit(a, b):
    qml.AngleEmbedding(features=a, wires=range(numQubits), rotation="X")
    qml.adjoint(qml.AngleEmbedding(features=b, wires=range(numQubits), rotation="X"))
    return qml.probs(wires=range(numQubits))

In [20]:
def qkernel(A, B):
    return np.array([[qkernel_circuit(a, b)[0]for b in B]for a in A])

In [21]:
scalar = StandardScaler()

X = scalar.fit_transform(myStock[['ds', 'volume']])
y = myStock['y'].values

In [22]:
x_train, y_train, x_test, y_test = X[:200], y[:200], X[200:250], y[200:250]

In [23]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((200, 2), (200,), (50, 2), (50,))

In [24]:
qsvr = SVR(kernel=qkernel)
qsvr.fit(x_train, y_train)

SVR(kernel=<function qkernel at 0x0000025D6BF85800>)

In [25]:
y_pred = qsvr.predict(x_test)

In [14]:
y_pred

array([223.95171779, 223.95815083, 224.03432184, 224.08645264,
       224.08412157, 224.07420887, 224.07800245, 224.10897569,
       224.01699659, 224.07760565, 224.08847442, 224.0458835 ,
       224.10575724, 224.10791753, 223.96674379, 224.07764248,
       224.11974388, 224.11599438, 224.1325325 , 224.10991834,
       224.1183688 , 224.08585256, 224.09657732, 224.0973226 ,
       224.11199898, 224.1175551 , 223.98500585, 224.02300124,
       224.11739345, 224.11612708, 224.10560298, 224.0818097 ,
       224.07639095, 224.03017514, 223.04269654, 223.97714017,
       224.00865856, 224.07279558, 224.01236362, 224.12032376,
       224.11447186, 224.11197936, 224.08151173, 224.05112208,
       223.98303898, 224.06809815, 224.06993898, 224.12113545,
       224.0899727 , 224.06300237])

In [15]:
y_test

array([222.91000366, 225.91000366, 230.1000061 , 233.66999817,
       233.3999939 , 231.41000366, 231.41000366, 230.57000732,
       230.75999451, 235.86000061, 236.47999573, 235.        ,
       232.1499939 , 231.77999878, 233.8500061 , 231.30000305,
       227.55000305, 227.55000305, 229.03999329, 229.53999329,
       225.77000427, 221.69000244, 226.80000305, 226.80000305,
       225.66999817, 226.77999878, 226.21000671, 233.        ,
       227.78999329, 227.78999329, 227.52000427, 226.36999512,
       227.36999512, 226.47000122, 228.19999695, 228.86999512,
       220.69000244, 216.78999329, 216.32000732, 222.5       ,
       222.5       , 222.77000427, 222.66000366, 220.11000061,
       220.91000366, 220.82000732, 220.82000732, 222.38000488,
       220.8500061 , 222.77000427])

In [16]:
# import pennylane as qml
# from pennylane import numpy as np

# n_qubits = 2
# dev = qml.device("default.qubit", wires=n_qubits)

# # Example parameterized circuit (same as kernel encoder)
# @qml.qnode(dev, interface="autograd")
# def circuit(params, x):
#     qml.AngleEmbedding(x, wires=range(n_qubits))
#     for i in range(n_qubits):
#         qml.RY(params[i], wires=i)
#     return qml.expval(qml.PauliZ(0))

# # Function to compute gradient variance
# def estimate_grad_variance(param_shape, n_param_samples=50, n_input_samples=20):
#     grad_fn = qml.grad(circuit, argnum=0)
#     grad_variances = []

#     for _ in range(n_param_samples):
#         params = np.random.uniform(0, 2 * np.pi, size=param_shape)
#         grads = []
#         for _ in range(n_input_samples):
#             x = np.random.uniform(0, np.pi, size=(n_qubits,))
#             g = grad_fn(params, x)
#             grads.append(np.array(g, dtype=float))
#         grads = np.stack(grads)
#         grad_variances.append(np.var(grads, axis=0))

#     grad_variances = np.stack(grad_variances)
#     mean_var = np.mean(grad_variances)
#     return grad_variances.mean(axis=0), mean_var


# param_shape = (n_qubits,)
# var_per_param, mean_var = estimate_grad_variance(param_shape)

# print("Per-parameter gradient variance:", var_per_param)
# print("Mean gradient variance:", mean_var)
